In [ ]:
%reset

In [ ]:
import os, sys

import pandas as pd
import numpy as np

import pacmagic
import pacmagic_icu

from importlib import reload
from data_warehouse_utils.dataloader import DataLoader

from causal_inference.experiment_generator.initialize_experiment import initialize_experiment
from causal_inference.experiment_generator.initialize_experiment import add_parameter
# from causal_inference.experiment_genertor.combiner import DataHandler #not ready yet

In [ ]:
# When running the notebook for the first time, change wd to
# the path of causal_inference folder
os.getcwd()
os.chdir('/home/adam/files/causal_inference')
os.getcwd()

In [ ]:
# Reloads packages

reload(sys.modules['causal_inference'])
reload(sys.modules['causal_inference.experiment_generator'])
reload(sys.modules['causal_inference.experiment_generator.initialize_experiment'])

from causal_inference.experiment_generator.initialize_experiment import initialize_experiment
from causal_inference.experiment_generator.initialize_experiment import add_parameter
from causal_inference.data_creator.combiner import DataHandler

In [ ]:
dl = DataLoader()

df = initialize_experiment(dl = dl,
                           n_of_patients = 5,
                           min_length_of_intubation = 24)

In [ ]:
test_df = pd.DataFrame(np.ones((10,10)))

test_df.head()

In [ ]:
#df.info()

In [ ]:
# doesn't work yet

#add_parameter(dl, df, 'fio2')
